<a href="https://colab.research.google.com/github/yvt-h/SCA_SOP/blob/main/SCA_HW5_Tailored_Sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Find annual operational cost (holding + ordering) for the optimal sourcing strategy if:
1. Products are sourced independently

In [1]:
from math import sqrt

products={
    1: {'D': 1000, 'S': 100, 's': 10, 'C': 50, 'h': 0.2},
    2: {'D': 300, 'S': 100, 's': 20, 'C': 60, 'h': 0.2},
    3: {'D': 100, 'S': 100, 's': 25, 'C': 30, 'h': 0.2},
    4: {'D': 50, 'S': 100, 's': 25, 'C': 30, 'h': 0.2}
}

results = {}
total_TC = 0

for product, details in products.items():
    D, S, s, C, h = details['D'], details['S'], details['s'], details['C'], details['h']
    Q = sqrt((2 * D * (S+s)) / (C * h))
    n = D / Q
    TC = (D * (S+s) / Q) + (Q * C * h / 2)
    total_TC += TC
    results[product] = {'Optimal Lot Size': Q, 'Order Frequency': n, 'Cost': TC}

results

{1: {'Optimal Lot Size': 148.32396974191326,
  'Order Frequency': 6.741998624632421,
  'Cost': 1483.2396974191327},
 2: {'Optimal Lot Size': 77.45966692414834,
  'Order Frequency': 3.872983346207417,
  'Cost': 929.51600308978},
 3: {'Optimal Lot Size': 64.54972243679029,
  'Order Frequency': 1.5491933384829666,
  'Cost': 387.2983346207417},
 4: {'Optimal Lot Size': 45.64354645876384,
  'Order Frequency': 1.0954451150103321,
  'Cost': 273.8612787525831}}

In [2]:
print('Annual Operational Cost:', total_TC)

Annual Operational Cost: 3073.9153138822376


2. All four products are sourced with the same frequency

In [3]:
from typing import no_type_check_decorator
# Calculating aggregated values
s_total = 100 + sum(product['s'] for product in products.values())
sum_DCh = sum(product['D'] * product['C'] * product['h'] for product in products.values())

# Ordering Cost
n = sqrt((sum_DCh) / (2 * s_total))
total_cost_order = n * s_total

# Holding Cost
results = {}
total_cost_holding = 0

for product, details in products.items():
    D, S, s, C, h = details['D'], details['S'], details['s'], details['C'], details['h']
    Q = D / n
    Cost_holding = C * h * Q / 2
    total_cost_holding += Cost_holding
    results[product] = {'Optimal Lot Size': Q, 'Order Frequency': n, 'Holding Cost': Cost_holding}

results

{1: {'Optimal Lot Size': 157.56771943166706,
  'Order Frequency': 6.346477588219924,
  'Holding Cost': 787.8385971583352},
 2: {'Optimal Lot Size': 47.27031582950012,
  'Order Frequency': 6.346477588219924,
  'Holding Cost': 283.6218949770007},
 3: {'Optimal Lot Size': 15.756771943166706,
  'Order Frequency': 6.346477588219924,
  'Holding Cost': 47.27031582950012},
 4: {'Optimal Lot Size': 7.878385971583353,
  'Order Frequency': 6.346477588219924,
  'Holding Cost': 23.63515791475006}}

In [4]:
total_TC = total_cost_order + total_cost_holding
print('Annual Operational Cost:', total_TC)

Annual Operational Cost: 2284.7319317591723


3. Order frequencies are determined according to the tailored aggregation strategy

1. Identify most frequently ordered product (assume independent
ordering): That would be product 1.
2. For all products other than 1 recompute frequencies using only
product specific ordering costs (i.e., use s, and set S=0)
❑ These would be 7.7 for Medpro and 2.4 for Heavypro
3. Identify frequency of other products as a multiple 𝑚𝑖 (assume only
product specific fixed cost) and round up
❑ mMedpro = roundup(11.0/7.7)=2, mHeavypro = roundup(11/2.4)=5
❑ Hence order Medpro every other time and Heavypro every 5th time
4. Recalculate ordering frequency n of most frequently ordered
product
5. Identify ordering frequency of all products, using 𝑛 and 𝑚𝑖

In [5]:
from math import ceil

# Max Frequency - Product 1 - 6.74 From Question 1
results = {}

for product, details in products.items():
  if product > 1:
    D, S, s, C, h = details['D'], details['S'], details['s'], details['C'], details['h']
    Q = sqrt((2 * D * (s)) / (C * h))
    n = D / Q
    m = ceil(6.74 / n)
    results[product] = {'Order Frequency': n, 'Order Frequency as as multiple of m': m}

results

{2: {'Order Frequency': 9.486832980505138,
  'Order Frequency as as multiple of m': 1},
 3: {'Order Frequency': 3.464101615137755,
  'Order Frequency as as multiple of m': 2},
 4: {'Order Frequency': 2.449489742783178,
  'Order Frequency as as multiple of m': 3}}

In [6]:
# Add m in products
m_values = [1, 1, 2, 3]
for product_id, m_value in zip(products.keys(), m_values):
    products[product_id]['m'] = m_value

# New Order Frequency
n_1 = sqrt(sum(product['D'] * product['C'] * product['h'] * product['m'] for product in products.values())/ (2 * (products[1]['S'] + sum(product['s'] / product['m'] for product in products.values()))))

for product_id in products:
    if product_id in [1, 2]:
        products[product_id]['n'] = n_1
    elif product_id == 3:
        products[product_id]['n'] = n_1/2
    elif product_id == 4:
        products[product_id]['n'] = n_1/3

s_total = 100 + sum(product['s'] / product['m'] for product in products.values())
total_cost_order = n_1 * s_total

results = {}
total_cost_holding = 0

for product, details in products.items():
    D, S, s, C, h, n = details['D'], details['S'], details['s'], details['C'], details['h'], details['n']
    Q = D / n
    Cost_holding = C * h * Q / 2
    total_cost_holding += Cost_holding
    results[product] = {'Optimal Lot Size': Q, 'Order Frequency': n, 'Holding Cost': Cost_holding}

results

{1: {'Optimal Lot Size': 138.61615117764455,
  'Order Frequency': 7.21416654195255,
  'Holding Cost': 693.0807558882227},
 2: {'Optimal Lot Size': 41.58484535329337,
  'Order Frequency': 7.21416654195255,
  'Holding Cost': 249.50907211976022},
 3: {'Optimal Lot Size': 27.723230235528913,
  'Order Frequency': 3.607083270976275,
  'Holding Cost': 83.16969070658674},
 4: {'Optimal Lot Size': 20.792422676646687,
  'Order Frequency': 2.40472218065085,
  'Holding Cost': 62.37726802994006}}

In [7]:
total_TC = total_cost_order + total_cost_holding
print('Annual Operational Cost:', total_TC)

Annual Operational Cost: 2176.2735734890193
